In [73]:
import pandas as pd
import numpy as np
import time
import os
import zipfile
import pandas_gbq
import glob
from google.cloud import bigquery
import re
from datetime import datetime
import locale

In [74]:
locale.setlocale(locale.LC_ALL, 'pt_BR')

'pt_BR'

## Aposentadoria gráfico 2

In [75]:
client = bigquery.Client()

In [76]:
query = """ 
        SELECT ANO as ano, MES as mes,  Id_SERVIDOR_PORTAL as id_servidor, TIPO_APOSENTADORIA as tipo_aposentadoria, DATA_APOSENTADORIA as data_aposentadoria, DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO as data_diploma_ingresso_servicopublico,
            CASE WHEN ORGSUP_LOTACAO ="Sem informação" THEN ORG_LOTACAO ELSE ORGSUP_LOTACAO 
            END as org_lotacao, 'REGIME JURIDICO UNICO' as regime_juridico,
            1 as total FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2023_nov_siape_aposentados_cadastro`
        """  

In [77]:
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')
df

Downloading: 100%|██████████|


,ano,mes,id_servidor,tipo_aposentadoria,data_aposentadoria,data_diploma_ingresso_servicopublico,org_lotacao,regime_juridico,total
0,2023,11,311786085,APOSENTADORIA POR INVALIDEZ,30/04/2008,12/08/1982,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1
1,2023,11,311413108,APOSENTADORIA VOLUNTARIA,07/04/2017,01/08/1981,MINISTERIO DA CULTURA,REGIME JURIDICO UNICO,1
2,2023,11,310796209,APOSENTADORIA VOLUNTARIA,12/04/2002,15/03/1990,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1
3,2023,11,313314589,APOSENTADORIA VOLUNTARIA,12/05/2011,01/10/1990,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1
4,2023,11,312332839,APOSENTADORIA VOLUNTARIA,14/06/2002,15/02/1977,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1
...,...,...,...,...,...,...,...,...,...
418496,2023,11,312385265,APOSENTADORIA VOLUNTARIA,30/09/2009,01/09/1979,Ministério da Defesa,REGIME JURIDICO UNICO,1
418497,2023,11,312133569,APOSENTADORIA VOLUNTARIA,02/04/2012,17/09/1980,Ministério da Defesa,REGIME JURIDICO UNICO,1
418498,2023,11,312305623,APOSENTADORIA VOLUNTARIA,06/09/1994,01/04/1984,Ministério da Defesa,REGIME JURIDICO UNICO,1
418499,2023,11,311654066,APOSENTADORIA POR INVALIDEZ,15/09/2009,01/09/1977,Ministério da Defesa,REGIME JURIDICO UNICO,1


In [78]:
df.columns

Index(['ano', 'mes', 'id_servidor', 'tipo_aposentadoria', 'data_aposentadoria',
       'data_diploma_ingresso_servicopublico', 'org_lotacao',
       'regime_juridico', 'total'],
      dtype='object')

In [79]:
df.dtypes

ano                                      Int64
mes                                      Int64
id_servidor                              Int64
tipo_aposentadoria                      object
data_aposentadoria                      object
data_diploma_ingresso_servicopublico    object
org_lotacao                             object
regime_juridico                         object
total                                    Int64
dtype: object

In [80]:
aposentadoria = df['data_aposentadoria'].isnull()
aposentadoria.value_counts()

False    417698
True        803
Name: data_aposentadoria, dtype: int64

In [81]:
ingresso = df['data_diploma_ingresso_servicopublico'].isnull()
ingresso.value_counts()

False    412512
True       5989
Name: data_diploma_ingresso_servicopublico, dtype: int64

In [82]:
df = df.dropna() 
df #Dados nulos da data de aposentadoria = 803; Dados nulos da data de ingresso = 5989. A soma dos dois é a quantidade de linhas dropadas (6792). 


,ano,mes,id_servidor,tipo_aposentadoria,data_aposentadoria,data_diploma_ingresso_servicopublico,org_lotacao,regime_juridico,total
0,2023,11,311786085,APOSENTADORIA POR INVALIDEZ,30/04/2008,12/08/1982,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1
1,2023,11,311413108,APOSENTADORIA VOLUNTARIA,07/04/2017,01/08/1981,MINISTERIO DA CULTURA,REGIME JURIDICO UNICO,1
2,2023,11,310796209,APOSENTADORIA VOLUNTARIA,12/04/2002,15/03/1990,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1
3,2023,11,313314589,APOSENTADORIA VOLUNTARIA,12/05/2011,01/10/1990,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1
4,2023,11,312332839,APOSENTADORIA VOLUNTARIA,14/06/2002,15/02/1977,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1
...,...,...,...,...,...,...,...,...,...
418496,2023,11,312385265,APOSENTADORIA VOLUNTARIA,30/09/2009,01/09/1979,Ministério da Defesa,REGIME JURIDICO UNICO,1
418497,2023,11,312133569,APOSENTADORIA VOLUNTARIA,02/04/2012,17/09/1980,Ministério da Defesa,REGIME JURIDICO UNICO,1
418498,2023,11,312305623,APOSENTADORIA VOLUNTARIA,06/09/1994,01/04/1984,Ministério da Defesa,REGIME JURIDICO UNICO,1
418499,2023,11,311654066,APOSENTADORIA POR INVALIDEZ,15/09/2009,01/09/1977,Ministério da Defesa,REGIME JURIDICO UNICO,1


In [83]:
df['data_aposentadoria'] = pd.to_datetime(df['data_aposentadoria'], dayfirst=True, format= "%d/%m/%Y") #transformando em data

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_2416\1474179940.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['data_aposentadoria'] = pd.to_datetime(df['data_aposentadoria'], dayfirst=True, format= "%d/%m/%Y") #transformando em data


In [84]:
df['data_diploma_ingresso_servicopublico'] = pd.to_datetime(df['data_diploma_ingresso_servicopublico'], dayfirst=True, format= "%d/%m/%Y") #transformando em data

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_2416\3069410005.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['data_diploma_ingresso_servicopublico'] = pd.to_datetime(df['data_diploma_ingresso_servicopublico'], dayfirst=True, format= "%d/%m/%Y") #transformando em data


In [85]:
df['ano_aposentadoria'] = df['data_aposentadoria'].dt.year #pegando apenas o ano

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_2416\1637080438.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ano_aposentadoria'] = df['data_aposentadoria'].dt.year #pegando apenas o ano


In [86]:
df['ano_ingresso'] = df['data_diploma_ingresso_servicopublico'].dt.year #pegando apenas o ano

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_2416\2359990388.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ano_ingresso'] = df['data_diploma_ingresso_servicopublico'].dt.year #pegando apenas o ano


In [87]:
df['ano_ingresso']= df['ano_ingresso'].astype(int) #transformando os valores da coluna em inteiro

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_2416\3392378587.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ano_ingresso']= df['ano_ingresso'].astype(int) #transformando os valores da coluna em inteiro


In [88]:
df['ano_ingresso'].unique()

array([1982, 1981, 1990, 1977, 1985, 1978, 1971, 1986, 1984, 1976, 1980,
       1974, 1972, 1989, 2013, 2014, 1979, 1983, 1987, 1991, 1963, 1954,
       1942, 1949, 1953, 1952, 1992, 2016, 1975, 1988, 1955, 1957, 1950,
       1960, 1956, 1947, 1945, 1959, 1967, 1966, 1965, 1970, 1996, 1995,
       1993, 2008, 1997, 1961, 1968, 1962, 1999, 1998, 1973, 1994, 1958,
       1964, 1969, 1946, 2004, 2006, 2007, 2009, 2011, 2012, 1943, 2010,
       1951, 2000, 1944, 2002, 2001, 2005, 2003, 1948, 1941, 2017, 2019,
       2015, 1939, 1940, 2018, 2020, 2021, 1936, 1932, 1935, 1938])

In [89]:
df['ano_aposentadoria'] = df['ano_aposentadoria'].astype(int) #transformando os valores da coluna em inteiro

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_2416\3254720995.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ano_aposentadoria'] = df['ano_aposentadoria'].astype(int) #transformando os valores da coluna em inteiro


In [90]:
df['ano_aposentadoria'].unique()

array([2008, 2017, 2002, 2011, 2000, 2012, 2005, 2004, 2003, 2014, 1988,
       2016, 1991, 2010, 2021, 2018, 1997, 2013, 2015, 1996, 2009, 2019,
       2023, 2022, 2020, 1999, 1987, 1994, 2001, 1992, 2006, 1995, 1986,
       1998, 1977, 1990, 1993, 2007, 1980, 1989, 1985, 1984, 1983, 1981,
       1982, 1979, 1976, 1978, 1965, 1968, 1972, 1971, 1970, 1966, 1967,
       1963, 1974, 1969, 1975, 1973, 1964, 1959, 1962, 1960, 1952, 1953,
       1955, 1958, 1954, 1951, 1961, 1957])

In [91]:
limites = [0, 1959, 1969, 1979, 1989, 1999, 2009, 2019, 2024]
categorias = ['Década de 50', 'Década de 60', 'Década de 70', 'Década de 80', 'Década de 90', 'Década de 2000', 'Década de 2010', 'Década de 2020']

df['decada_aposentadoria'] = pd.cut(df['ano_aposentadoria'], bins=limites, labels=categorias)


C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_2416\1645037803.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['decada_aposentadoria'] = pd.cut(df['ano_aposentadoria'], bins=limites, labels=categorias)


In [92]:
df = df[['ano', 'mes', 'tipo_aposentadoria', 'data_aposentadoria', 'ano_aposentadoria', 'decada_aposentadoria', 'data_diploma_ingresso_servicopublico', 'ano_ingresso', 'org_lotacao', 'regime_juridico', 'total']]
df #411709 linhdas

,ano,mes,tipo_aposentadoria,data_aposentadoria,ano_aposentadoria,decada_aposentadoria,data_diploma_ingresso_servicopublico,ano_ingresso,org_lotacao,regime_juridico,total
0,2023,11,APOSENTADORIA POR INVALIDEZ,2008-04-30,2008,Década de 2000,1982-08-12,1982,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1
1,2023,11,APOSENTADORIA VOLUNTARIA,2017-04-07,2017,Década de 2010,1981-08-01,1981,MINISTERIO DA CULTURA,REGIME JURIDICO UNICO,1
2,2023,11,APOSENTADORIA VOLUNTARIA,2002-04-12,2002,Década de 2000,1990-03-15,1990,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1
3,2023,11,APOSENTADORIA VOLUNTARIA,2011-05-12,2011,Década de 2010,1990-10-01,1990,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1
4,2023,11,APOSENTADORIA VOLUNTARIA,2002-06-14,2002,Década de 2000,1977-02-15,1977,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1
...,...,...,...,...,...,...,...,...,...,...,...
418496,2023,11,APOSENTADORIA VOLUNTARIA,2009-09-30,2009,Década de 2000,1979-09-01,1979,Ministério da Defesa,REGIME JURIDICO UNICO,1
418497,2023,11,APOSENTADORIA VOLUNTARIA,2012-04-02,2012,Década de 2010,1980-09-17,1980,Ministério da Defesa,REGIME JURIDICO UNICO,1
418498,2023,11,APOSENTADORIA VOLUNTARIA,1994-09-06,1994,Década de 90,1984-04-01,1984,Ministério da Defesa,REGIME JURIDICO UNICO,1
418499,2023,11,APOSENTADORIA POR INVALIDEZ,2009-09-15,2009,Década de 2000,1977-09-01,1977,Ministério da Defesa,REGIME JURIDICO UNICO,1


In [93]:
df['tempo_contribuicao_servico_publico'] = df['ano_aposentadoria'] - df['ano_ingresso']
df['tempo_contribuicao_servico_publico']

0         26
1         36
2         12
3         21
4         25
          ..
418496    30
418497    32
418498    10
418499    32
418500    18
Name: tempo_contribuicao_servico_publico, Length: 411709, dtype: int32

In [94]:
df['tempo_contribuicao_servico_publico'].unique()

array([ 26,  36,  12,  21,  25,  15,  22,  41,  19,  18,  24,  32,  34,
        11,  30,  38,  17,  20,  31,  40,  28,   0,  23,  27,   2,  14,
        33,  16,  39,  29,  46,  55,  67,  60,  48,  56,  57,  42,   5,
        37,  43,  13,  44,  45,   1,   7,   3,   4,   8,  35,  50,  47,
        54,  49,   6,  52,  10,  -8,   9,  53,  66,  -5, -27,  -1,  -2,
       -12,  -3,  58, -13, -14,  51,  -6, -16, -21,  -7,  59,  -9,  -4,
        61,  65,  62, -17,  64,  63, -24, -11, -10, -15, -18, -19, -22])

In [95]:
df_zeros = df[df['tempo_contribuicao_servico_publico']==0] #1338 linhas
df_zeros

,ano,mes,tipo_aposentadoria,data_aposentadoria,ano_aposentadoria,decada_aposentadoria,data_diploma_ingresso_servicopublico,ano_ingresso,org_lotacao,regime_juridico,total,tempo_contribuicao_servico_publico
28,2023,11,APOSENTADORIA VOLUNTARIA,2013-01-02,2013,Década de 2010,2013-01-01,2013,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,0
29,2023,11,APOSENTADORIA VOLUNTARIA,2013-01-02,2013,Década de 2010,2013-01-01,2013,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,0
32,2023,11,APOSENTADORIA VOLUNTARIA,2014-06-05,2014,Década de 2010,2014-06-04,2014,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,0
34,2023,11,APOSENTADORIA VOLUNTARIA,2014-06-05,2014,Década de 2010,2014-06-04,2014,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,0
36,2023,11,APOSENTADORIA VOLUNTARIA,2014-06-05,2014,Década de 2010,2014-06-04,2014,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
411250,2023,11,APOSENTADORIA VOLUNTARIA,1983-04-06,1983,Década de 80,1983-04-04,1983,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,0
412141,2023,11,APOSENTADORIA POR INVALIDEZ,1973-01-25,1973,Década de 70,1973-01-11,1973,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,0
413399,2023,11,APOSENTADORIA VOLUNTARIA,1997-08-08,1997,Década de 90,1997-08-01,1997,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,0
414344,2023,11,APOSENTADORIA VOLUNTARIA,1998-12-02,1998,Década de 90,1998-06-26,1998,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,0


In [96]:
menor_zero = df[df['tempo_contribuicao_servico_publico']< 0] #242 linhas
menor_zero 

,ano,mes,tipo_aposentadoria,data_aposentadoria,ano_aposentadoria,decada_aposentadoria,data_diploma_ingresso_servicopublico,ano_ingresso,org_lotacao,regime_juridico,total,tempo_contribuicao_servico_publico
1156,2023,11,APOSENTADORIA VOLUNTARIA,1983-04-06,1983,Década de 80,1991-01-02,1991,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,-8
25447,2023,11,APOSENTADORIA VOLUNTARIA,1995-05-09,1995,Década de 90,2000-04-01,2000,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,-5
27835,2023,11,APOSENTADORIA POR INVALIDEZ,1966-12-01,1966,Década de 60,1993-11-01,1993,"Ministério da Ciência, Tecnologia, Inovações e...",REGIME JURIDICO UNICO,1,-27
27837,2023,11,APOSENTADORIA VOLUNTARIA,1979-05-28,1979,Década de 70,1980-04-23,1980,"Ministério da Ciência, Tecnologia, Inovações e...",REGIME JURIDICO UNICO,1,-1
27854,2023,11,APOSENTADORIA POR INVALIDEZ,1978-10-16,1978,Década de 70,1980-04-01,1980,"Ministério da Ciência, Tecnologia, Inovações e...",REGIME JURIDICO UNICO,1,-2
...,...,...,...,...,...,...,...,...,...,...,...,...
412537,2023,11,APOSENTADORIA VOLUNTARIA,1992-07-31,1992,Década de 90,1997-07-01,1997,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,-5
412614,2023,11,APOSENTADORIA VOLUNTARIA,1969-04-02,1969,Década de 60,1991-01-31,1991,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,-22
413024,2023,11,APOSENTADORIA VOLUNTARIA,1996-06-14,1996,Década de 90,1997-07-01,1997,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,-1
413216,2023,11,APOSENTADORIA VOLUNTARIA,1989-06-23,1989,Década de 80,1991-01-31,1991,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,-2


In [97]:
df = df[df['tempo_contribuicao_servico_publico'] > 0]
df['tempo_contribuicao_servico_publico'].unique()

array([26, 36, 12, 21, 25, 15, 22, 41, 19, 18, 24, 32, 34, 11, 30, 38, 17,
       20, 31, 40, 28, 23, 27,  2, 14, 33, 16, 39, 29, 46, 55, 67, 60, 48,
       56, 57, 42,  5, 37, 43, 13, 44, 45,  1,  7,  3,  4,  8, 35, 50, 47,
       54, 49,  6, 52, 10,  9, 53, 66, 58, 51, 59, 61, 65, 62, 64, 63])

In [98]:
df.info() #Foram descartados as 1580 linhas que possuiam tempo de contribuição menor ou igual a zero.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 410129 entries, 0 to 418500
Data columns (total 12 columns):
 #   Column                                Non-Null Count   Dtype         
---  ------                                --------------   -----         
 0   ano                                   410129 non-null  Int64         
 1   mes                                   410129 non-null  Int64         
 2   tipo_aposentadoria                    410129 non-null  object        
 3   data_aposentadoria                    410129 non-null  datetime64[ns]
 4   ano_aposentadoria                     410129 non-null  int32         
 5   decada_aposentadoria                  410129 non-null  category      
 6   data_diploma_ingresso_servicopublico  410129 non-null  datetime64[ns]
 7   ano_ingresso                          410129 non-null  int32         
 8   org_lotacao                           410129 non-null  object        
 9   regime_juridico                       410129 non-null  obje

In [99]:
df

,ano,mes,tipo_aposentadoria,data_aposentadoria,ano_aposentadoria,decada_aposentadoria,data_diploma_ingresso_servicopublico,ano_ingresso,org_lotacao,regime_juridico,total,tempo_contribuicao_servico_publico
0,2023,11,APOSENTADORIA POR INVALIDEZ,2008-04-30,2008,Década de 2000,1982-08-12,1982,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,26
1,2023,11,APOSENTADORIA VOLUNTARIA,2017-04-07,2017,Década de 2010,1981-08-01,1981,MINISTERIO DA CULTURA,REGIME JURIDICO UNICO,1,36
2,2023,11,APOSENTADORIA VOLUNTARIA,2002-04-12,2002,Década de 2000,1990-03-15,1990,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,12
3,2023,11,APOSENTADORIA VOLUNTARIA,2011-05-12,2011,Década de 2010,1990-10-01,1990,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,21
4,2023,11,APOSENTADORIA VOLUNTARIA,2002-06-14,2002,Década de 2000,1977-02-15,1977,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,25
...,...,...,...,...,...,...,...,...,...,...,...,...
418496,2023,11,APOSENTADORIA VOLUNTARIA,2009-09-30,2009,Década de 2000,1979-09-01,1979,Ministério da Defesa,REGIME JURIDICO UNICO,1,30
418497,2023,11,APOSENTADORIA VOLUNTARIA,2012-04-02,2012,Década de 2010,1980-09-17,1980,Ministério da Defesa,REGIME JURIDICO UNICO,1,32
418498,2023,11,APOSENTADORIA VOLUNTARIA,1994-09-06,1994,Década de 90,1984-04-01,1984,Ministério da Defesa,REGIME JURIDICO UNICO,1,10
418499,2023,11,APOSENTADORIA POR INVALIDEZ,2009-09-15,2009,Década de 2000,1977-09-01,1977,Ministério da Defesa,REGIME JURIDICO UNICO,1,32


## Tratamento final

In [100]:
df['tipo_aposentadoria'].unique()

array(['APOSENTADORIA POR INVALIDEZ', 'APOSENTADORIA VOLUNTARIA',
       'APOSENTADORIA COMPULSORIA', 'APOSENTADORIA POR INCAPACIDADE',
       'OUTROS'], dtype=object)

In [101]:
def renomear(var_aposentadoria):
    var_aposentadoria = re.sub(r"\bAPOSENTADORIA POR INVALIDEZ\b", "Aposentadoria por invalidez", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bAPOSENTADORIA VOLUNTARIA\b", "Aposentadoria voluntária", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bAPOSENTADORIA COMPULSORIA\b", "Aposentadoria compulsória", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bAPOSENTADORIA POR INCAPACIDADE\b", "Aposentadoria por incapacidade", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bOUTROS\b", "Outros", var_aposentadoria)
    
    return var_aposentadoria

In [102]:
df['tipo_aposentadoria'] = df['tipo_aposentadoria'].apply(renomear) 

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_2416\2971334208.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tipo_aposentadoria'] = df['tipo_aposentadoria'].apply(renomear)


In [103]:
df['tipo_aposentadoria'].unique()

array(['Aposentadoria por invalidez', 'Aposentadoria voluntária',
       'Aposentadoria compulsória', 'Aposentadoria por incapacidade',
       'Outros'], dtype=object)

In [104]:
df['org_lotacao'] = df['org_lotacao'].str.title()

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_2416\2304693947.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['org_lotacao'] = df['org_lotacao'].str.title()


In [105]:
df['org_lotacao'].unique()

array(['Min Gestao E Inov Em Serv Publicos', 'Ministerio Da Cultura',
       'Ministério Da Saúde', 'Ministério Da Defesa',
       'Ministério Da Educação', 'Ministerio Dos Povos Indigenas',
       'Presidência Da República', 'Ministério De Minas E Energia',
       'Ministério Da Justiça E Segurança Pública',
       'Ministério Da Ciência, Tecnologia, Inovações E Comunicações',
       'Ministerio Dos Transportes', 'Ministerio Da Previdencia Social',
       'Ministerio Do Planejamento E Orcamento', 'Ministerio Da Fazenda',
       'Ministério Das Relações Exteriores',
       'Ministério Da Agricultura, Pecuária E Abastecimento',
       'Min Da Integ E Do Desenv Regional',
       'Min Do Desenv Agr E Agric Familiar',
       'Ministerio Do Meio Ambiente', 'Ministério Do Turismo',
       'Ministerio Do Trabalho E Emprego',
       'Ministerio De Portos E Aeroportos',
       'Min Desenvolv Ind Comercio E Servicos',
       'Controladoria-Geral Da União', 'Ministerio Das Comunicacoes',
       '

In [106]:
def renomear2(var_aposentadoria):
    var_aposentadoria = re.sub(r"\bMin Gestao E Inov Em Serv Publicos\b", "Ministério Gestão E Inovação Em Serviços Públicos", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMinisterio Dos Povos Indigenas\b", "Ministário dos Povos Indígenas", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMinisterio Da Previdencia Social\b", "Ministério Da Previdência Social", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMinisterio Do Planejamento E Orcamento\b", "Ministério Do Planejamento E Orçamento", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMin Da Integ E Do Desenv Regional\b", "Ministério da Integração e do Desenvolvimento Regional", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMin Do Desenv Agr E Agric Familiar\b", "Ministério do Desenvolvimento Agrário e Agricultura Familiar", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMin Desenvolv Ind Comercio E Servicos\b", "Ministério do Desenvolvimento, Indústria, Comércio e Serviços", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bComunicacoes\b", "Comunicações", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMinisterio\b", "Ministério", var_aposentadoria)

    
    return var_aposentadoria

In [107]:
df['org_lotacao'] = df['org_lotacao'].apply(renomear2)

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_2416\3127125097.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['org_lotacao'] = df['org_lotacao'].apply(renomear2)


In [108]:
def renomear3(var_regime):
    var_regime = re.sub(r"\bREGIME JURIDICO UNICO\b", "Regime Jurídico Único", var_regime)
    return var_regime

In [109]:
df['regime_juridico'] = df['regime_juridico'].apply(renomear3)

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_2416\283232767.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['regime_juridico'] = df['regime_juridico'].apply(renomear3)


In [110]:
df['org_lotacao'].unique()

array(['Ministério Gestão E Inovação Em Serviços Públicos',
       'Ministério Da Cultura', 'Ministério Da Saúde',
       'Ministério Da Defesa', 'Ministério Da Educação',
       'Ministário dos Povos Indígenas', 'Presidência Da República',
       'Ministério De Minas E Energia',
       'Ministério Da Justiça E Segurança Pública',
       'Ministério Da Ciência, Tecnologia, Inovações E Comunicações',
       'Ministério Dos Transportes', 'Ministério Da Previdência Social',
       'Ministério Do Planejamento E Orçamento', 'Ministério Da Fazenda',
       'Ministério Das Relações Exteriores',
       'Ministério Da Agricultura, Pecuária E Abastecimento',
       'Ministério da Integração e do Desenvolvimento Regional',
       'Ministério do Desenvolvimento Agrário e Agricultura Familiar',
       'Ministério Do Meio Ambiente', 'Ministério Do Turismo',
       'Ministério Do Trabalho E Emprego',
       'Ministério De Portos E Aeroportos',
       'Ministério do Desenvolvimento, Indústria, Comérci

In [111]:
COLOCAR LETRAS INICAIS EM CAPS, subir dnv p gbq 
ministerios = {
    "Ministério Gestão E Inovação Em Serviços Públicos":"Economia, finanças, planejamento, previdência e gestão",
    "Ministério Da Cultura":"Cultura, turismo e esportes",
    "Ministério Da Saúde":"Saúde",
    "Ministério Da Defesa":"Justiça, segurança e órgãos de controle e defesa",
    "Ministério Da Educação":"Educação e Ciência e tecnologia",
    "Ministário dos Povos Indígenas":"Desenvolvimento Social, previdência e direitos humanos",
    "Presidência Da República": "Presidência Da República",
    "Ministério De Minas E Energia":"Infraestrutura, transporte, desenvolvimento regional, portos e aeroportos, minas e energia",
    "Ministério Da Justiça E Segurança Pública": "Justiça, segurança e órgãos de controle e defesa",
    "Ministério Da Ciência, Tecnologia, Inovações E Comunicações": "Educação e Ciência e tecnologia",
    "Ministério Dos Transportes": "Infraestrutura, transporte, desenvolvimento regional, portos e aeroportos, minas e energia",
    "Ministério Da Previdência Social": "Desenvolvimento Social, previdência e direitos humanos",
    "Ministério Do Planejamento E Orçamento": "Economia, finanças, planejamento e gestão",
    "Ministério Da Fazenda": "Economia, finanças, planejamento e gestão",
    "Ministério Das Relações Exteriores": "Relações exteriores",
    "Ministério Da Agricultura, Pecuária E Abastecimento": "Meio ambiente, agricultura e agropecuária",
    "Ministério da Integração e do Desenvolvimento Regional": "Infraestrutura, transporte, desenvolvimento regional, portos e aeroportos, minas e energia",
    "Ministério do Desenvolvimento Agrário e Agricultura Familiar": "Meio ambiente, agricultura e agropecuária",
    "Ministério Do Meio Ambiente": "Meio ambiente, agricultura e agropecuária",
    "Ministério Do Turismo": "Cultura, turismo e esportes",
    "Ministério Do Trabalho E Emprego": "Economia, finanças, planejamento e gestão",
    "Ministério De Portos E Aeroportos": "Infraestrutura, transporte, desenvolvimento regional, portos e aeroportos, minas e energia",
    "Ministério do Desenvolvimento, Indústria, Comércio e Serviços": "Infraestrutura, transporte, desenvolvimento regional, portos e aeroportos, minas e energia",
    "Controladoria-Geral Da União": "Justiça, segurança e órgãos de controle e defesa",
    "Ministério Das Comunicações": "Infraestrutura, transporte, desenvolvimento regional, portos e aeroportos, minas e energia",
    "Ministério Do Trabalho E Previdencia" : "Desenvolvimento Social, previdência e direitos humanos",
    "Defensoria Pública Da União": "Justiça, segurança e órgãos de controle e defesa",
    "Ministério Da Economia": "Economia, finanças, planejamento e gestão"
}

In [112]:
def categorizando(x):
    if x in ministerios:
        return ministerios[x]


In [113]:
df['categoria_ministerios'] = df['org_lotacao'].apply(categorizando)

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_2416\3148416238.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['categoria_ministerios'] = df['org_lotacao'].apply(categorizando)


In [114]:
df['categoria_ministerios'].unique()

array(['Economia, finanças, planejamento e gestão',
       'Cultura, turismo e esportes', 'Saúde',
       'Justiça, segurança e órgãos de controle e defesa',
       'Educação e Ciência e tecnologia',
       'Desenvolvimento Social, previdência e direitos humanos',
       'Presidência Da República',
       'Infraestrutura, transporte, desenvolvimento regional, portos e aeroportos, minas e energia',
       'Relações exteriores', 'Meio ambiente, agricultura e agropecuária'],
      dtype=object)

In [117]:
df['media_por_decada_aposentadoria'] = df.groupby('decada_aposentadoria')['tempo_contribuicao_servico_publico'].transform('mean')
#os valores na coluna 'tempo_contribuicao_servico_publico' são substituídos pelas médias de seus respectivos grupos, ou seja, as décadas.

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_2416\3127172447.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['media_por_decada_aposentadoria'] = df.groupby('decada_aposentadoria')['tempo_contribuicao_servico_publico'].transform('mean')


In [139]:
exemplo = df.groupby('decada_aposentadoria')['tempo_contribuicao_servico_publico'].mean().reset_index()
exemplo

,decada_aposentadoria,tempo_contribuicao_servico_publico
0,Década de 50,5.785714
1,Década de 60,10.880952
2,Década de 70,19.377212
3,Década de 80,26.318909
4,Década de 90,21.969950
5,Década de 2000,25.959999
6,Década de 2010,31.820581
7,Década de 2020,35.094978


In [133]:
df

,ano,mes,tipo_aposentadoria,data_aposentadoria,ano_aposentadoria,decada_aposentadoria,data_diploma_ingresso_servicopublico,ano_ingresso,org_lotacao,categoria_ministerios,regime_juridico,tempo_contribuicao_servico_publico,total,media_por_decada_aposentadoria
0,2023,11,Aposentadoria por invalidez,2008-04-30,2008,Década de 2000,1982-08-12,1982,Ministério Gestão E Inovação Em Serviços Públicos,"Economia, finanças, planejamento e gestão",Regime Jurídico Único,26,1,25.959999
1,2023,11,Aposentadoria voluntária,2017-04-07,2017,Década de 2010,1981-08-01,1981,Ministério Da Cultura,"Cultura, turismo e esportes",Regime Jurídico Único,36,1,31.820581
2,2023,11,Aposentadoria voluntária,2002-04-12,2002,Década de 2000,1990-03-15,1990,Ministério Gestão E Inovação Em Serviços Públicos,"Economia, finanças, planejamento e gestão",Regime Jurídico Único,12,1,25.959999
3,2023,11,Aposentadoria voluntária,2011-05-12,2011,Década de 2010,1990-10-01,1990,Ministério Gestão E Inovação Em Serviços Públicos,"Economia, finanças, planejamento e gestão",Regime Jurídico Único,21,1,31.820581
4,2023,11,Aposentadoria voluntária,2002-06-14,2002,Década de 2000,1977-02-15,1977,Ministério Gestão E Inovação Em Serviços Públicos,"Economia, finanças, planejamento e gestão",Regime Jurídico Único,25,1,25.959999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418496,2023,11,Aposentadoria voluntária,2009-09-30,2009,Década de 2000,1979-09-01,1979,Ministério Da Defesa,"Justiça, segurança e órgãos de controle e defesa",Regime Jurídico Único,30,1,25.959999
418497,2023,11,Aposentadoria voluntária,2012-04-02,2012,Década de 2010,1980-09-17,1980,Ministério Da Defesa,"Justiça, segurança e órgãos de controle e defesa",Regime Jurídico Único,32,1,31.820581
418498,2023,11,Aposentadoria voluntária,1994-09-06,1994,Década de 90,1984-04-01,1984,Ministério Da Defesa,"Justiça, segurança e órgãos de controle e defesa",Regime Jurídico Único,10,1,21.969950
418499,2023,11,Aposentadoria por invalidez,2009-09-15,2009,Década de 2000,1977-09-01,1977,Ministério Da Defesa,"Justiça, segurança e órgãos de controle e defesa",Regime Jurídico Único,32,1,25.959999


In [144]:
df[df['decada_aposentadoria']=='Década de 2000']

,ano,mes,regime_juridico,org_lotacao,categoria_ministerios,data_diploma_ingresso_servicopublico,ano_ingresso,tipo_aposentadoria,data_aposentadoria,ano_aposentadoria,decada_aposentadoria,media_por_decada_aposentadoria,tempo_contribuicao_servico_publico,total
0,2023,11,Regime Jurídico Único,Ministério Gestão E Inovação Em Serviços Públicos,"Economia, finanças, planejamento e gestão",1982-08-12,1982,Aposentadoria por invalidez,2008-04-30,2008,Década de 2000,25.959999,26,1
2,2023,11,Regime Jurídico Único,Ministério Gestão E Inovação Em Serviços Públicos,"Economia, finanças, planejamento e gestão",1990-03-15,1990,Aposentadoria voluntária,2002-04-12,2002,Década de 2000,25.959999,12,1
4,2023,11,Regime Jurídico Único,Ministério Gestão E Inovação Em Serviços Públicos,"Economia, finanças, planejamento e gestão",1977-02-15,1977,Aposentadoria voluntária,2002-06-14,2002,Década de 2000,25.959999,25,1
5,2023,11,Regime Jurídico Único,Ministério Gestão E Inovação Em Serviços Públicos,"Economia, finanças, planejamento e gestão",1985-02-04,1985,Aposentadoria voluntária,2000-05-12,2000,Década de 2000,25.959999,15,1
6,2023,11,Regime Jurídico Único,Ministério Gestão E Inovação Em Serviços Públicos,"Economia, finanças, planejamento e gestão",1977-02-15,1977,Aposentadoria voluntária,2002-10-23,2002,Década de 2000,25.959999,25,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418465,2023,11,Regime Jurídico Único,Ministério Gestão E Inovação Em Serviços Públicos,"Economia, finanças, planejamento e gestão",1980-02-27,1980,Aposentadoria voluntária,2005-09-23,2005,Década de 2000,25.959999,25,1
418466,2023,11,Regime Jurídico Único,Ministério Gestão E Inovação Em Serviços Públicos,"Economia, finanças, planejamento e gestão",1980-04-25,1980,Aposentadoria voluntária,2003-01-22,2003,Década de 2000,25.959999,23,1
418471,2023,11,Regime Jurídico Único,Ministério Do Planejamento E Orçamento,"Economia, finanças, planejamento e gestão",1984-06-15,1984,Aposentadoria compulsória,2002-12-03,2002,Década de 2000,25.959999,18,1
418496,2023,11,Regime Jurídico Único,Ministério Da Defesa,"Justiça, segurança e órgãos de controle e defesa",1979-09-01,1979,Aposentadoria voluntária,2009-09-30,2009,Década de 2000,25.959999,30,1


In [142]:
df = df[['ano', 'mes', 'regime_juridico', 'org_lotacao', 'categoria_ministerios', 'data_diploma_ingresso_servicopublico', 'ano_ingresso' ,'tipo_aposentadoria', 'data_aposentadoria', 'ano_aposentadoria', 'decada_aposentadoria', 'media_por_decada_aposentadoria',
       'tempo_contribuicao_servico_publico', 'total']]

In [143]:
df.columns

Index(['ano', 'mes', 'regime_juridico', 'org_lotacao', 'categoria_ministerios',
       'data_diploma_ingresso_servicopublico', 'ano_ingresso',
       'tipo_aposentadoria', 'data_aposentadoria', 'ano_aposentadoria',
       'decada_aposentadoria', 'media_por_decada_aposentadoria',
       'tempo_contribuicao_servico_publico', 'total'],
      dtype='object')

In [47]:
client = bigquery.Client()
dataset_ref = client.dataset('perfil_remuneracao')

In [48]:
df.columns

Index(['ano', 'mes', 'tipo_aposentadoria', 'data_aposentadoria',
       'ano_aposentadoria', 'decada_aposentadoria',
       'data_diploma_ingresso_servicopublico', 'ano_ingresso', 'org_lotacao',
       'categoria_ministerios', 'regime_juridico',
       'tempo_contribuicao_servico_publico', 'total'],
      dtype='object')

In [53]:
df['org_lotacao'].unique()

array(['Ministério Gestão E Inovação Em Serviços Públicos',
       'Ministério Da Cultura', 'Ministério Da Saúde',
       'Ministério Da Defesa', 'Ministério Da Educação',
       'Ministário dos Povos Indígenas', 'Presidência Da República',
       'Ministério De Minas E Energia',
       'Ministério Da Justiça E Segurança Pública',
       'Ministério Da Ciência, Tecnologia, Inovações E Comunicações',
       'Ministério Dos Transportes', 'Ministério Da Previdência Social',
       'Ministério Do Planejamento E Orçamento', 'Ministério Da Fazenda',
       'Ministério Das Relações Exteriores',
       'Ministério Da Agricultura, Pecuária E Abastecimento',
       'Ministério da Integração e do Desenvolvimento Regional',
       'Ministério do Desenvolvimento Agrário e Agricultura Familiar',
       'Ministério Do Meio Ambiente', 'Ministério Do Turismo',
       'Ministério Do Trabalho E Emprego',
       'Ministério De Portos E Aeroportos',
       'Ministério do Desenvolvimento, Indústria, Comérci

In [145]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 410129 entries, 0 to 418500
Data columns (total 14 columns):
 #   Column                                Non-Null Count   Dtype         
---  ------                                --------------   -----         
 0   ano                                   410129 non-null  Int64         
 1   mes                                   410129 non-null  Int64         
 2   regime_juridico                       410129 non-null  object        
 3   org_lotacao                           410129 non-null  object        
 4   categoria_ministerios                 410129 non-null  object        
 5   data_diploma_ingresso_servicopublico  410129 non-null  datetime64[ns]
 6   ano_ingresso                          410129 non-null  int32         
 7   tipo_aposentadoria                    410129 non-null  object        
 8   data_aposentadoria                    410129 non-null  datetime64[ns]
 9   ano_aposentadoria                     410129 non-null  int3

In [146]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referência da observação'),
        bigquery.SchemaField('mes','INTEGER',description='Mês de referência da observação'),  
        bigquery.SchemaField('regime_juridico','STRING',description='Regime jurídico'),
        bigquery.SchemaField('org_lotacao','STRING',description='Órgão de lotação'),
        bigquery.SchemaField('categoria_ministerios','STRING',description='Categorias dos ministérios'),
        bigquery.SchemaField('data_diploma_ingresso_servicopublico','DATETIME',description='Data de ingresso do servidor'),
        bigquery.SchemaField('ano_ingresso','INTEGER',description='Ano de ingresso do servidor'),
        bigquery.SchemaField('tipo_aposentadoria','STRING',description='Tipo de aposentadoria'),
        bigquery.SchemaField('data_aposentadoria','DATETIME',description='Data da aposentadoria'),
        bigquery.SchemaField('ano_aposentadoria','INTEGER',description='Ano de aposentadoria'),
        bigquery.SchemaField('decada_aposentadoria','STRING',description='Década da aposentadoria'),
        bigquery.SchemaField('media_por_decada_aposentadoria','FLOAT',description='Década da aposentadoria'),
        bigquery.SchemaField('tempo_contribuicao_servico_publico','INTEGER',description='Tempo de contribuição no serviço público'),
        bigquery.SchemaField('total','INTEGER',description='Quantidade total da observação')]


In [147]:
table_ref = dataset_ref.table('SIAPE_ingresso_aposentadoria_media') 
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result() 


LoadJob<project=repositoriodedadosgpsp, location=US, id=588d54c9-aeba-47db-b077-9552ae231897>